In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import mean_absolute_error

from tinymlgen import port

import numpy as np
from numpy import asarray

from utils.preprocessing import preprocessImage
from utils.grad_cam import get_img_array, make_gradcam_heatmap, save_and_display_gradcam
from utils.image_display import imageDisplay
from utils.transform_data import transformData

In [2]:
#Preprocessing
resize_image = True
#new_size = (480, 640)
new_size = (640, 480)
grey_scale = True
rgb_or_grey = 3
normalize_image = False
thresholding = False
erosion = False
kernel = np.ones((2, 2), np.uint8)
cut_image = False
turn_image = False

#Scaling
min_max = False
z_score = False

min_max_labels = False
z_score_labels = False

#Training
batch_size = 16
epochs = 150

if resize_image == True:
    #new_size = (300, 225)
    #new_size = (225, 300)
    if turn_image == True:
        #new_size = (160, 120)
        new_size = (120, 160)
        #new_size = (240, 320)
    else:
        new_size = (160, 120)
        #new_size = (320, 240)

if grey_scale == True:
    rgb_or_grey = 1

preprocessImage = preprocessImage()

In [3]:
imageFoldersValidation = "../Bilder/Manometer/Blindset final test"

#Load in Images and Clean DataFrame
validation_images, validation_labels = preprocessImage.load_images(imageFoldersValidation, True, grey_scale, resize_image, new_size, normalize_image, thresholding, erosion, kernel, cut_image, turn_image)

validation_images = validation_images.reshape(len(validation_labels), new_size[1], new_size[0], rgb_or_grey)
nRows,nCols,nDims = validation_images.shape[1:]
validation_images = validation_images.reshape(validation_images.shape[0], nRows, nCols, nDims)
#Scale Data
transformer = transformData()

#Parameters Training data
mean_value = 114.31817292906746
standard_deviation = 77.30169303183406

min_value = 0
max_value = 255

#Parameters Targetr Data
mean_value_labels = 5.035714285714286
standard_deviation_labels = 3.0150275548430043

min_labels = 0
max_labels = 10


if min_max == True:
    validation_images = transformer.scale_min_max(validation_images, min_value, max_value)

if z_score == True:
    validation_images = transformer.scale_z_score(validation_images, mean_value, standard_deviation)


if min_max_labels == True:
    validation_labels = transformer.scale_min_max(validation_labels, min_labels, max_labels)

if z_score_labels == True:
    validation_labels = transformer.scale_z_score(validation_labels, mean_value_labels, standard_deviation_labels)


imageFolderPath: ../Bilder/Manometer/Blindset final test


In [4]:
image_num = len(validation_labels)-1

path = "models/model2/model.tflite"

from sklearn.metrics import mean_squared_error

preds = []

interpreter = tf.lite.Interpreter(model_path=path)
interpreter.allocate_tensors()

for element in range(len(validation_labels)):
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_data = validation_images[element].reshape(1, 120, 160, 1)#.flatten()
    input_data = np.float32(input_data)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    predictions=interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    preds.append(output_data[0])

dict = {"True Value":"Predction"}
for A, B in zip(validation_labels, preds):
    dict[A] = B[0]

dict


{'True Value': 'Predction',
 0.3: 0.5912309,
 0.5: 0.94844514,
 0.8: 0.94844514,
 0.0: 0.3405181,
 1.3: 1.3779117,
 1.5: 1.7255563,
 1.8: 1.9736549,
 1.0: 1.081826,
 10.0: 9.2741375,
 2.3: 2.2617373,
 2.5: 2.4233904,
 2.8: 2.70203,
 2.0: 2.0484233,
 3.3: 3.2267604,
 3.5: 3.4911416,
 3.8: 3.7960181,
 3.0: 2.8444428,
 4.3: 3.9012382,
 4.5: 3.9978817,
 4.8: 4.3437223,
 4.0: 3.728656,
 5.3: 4.883539,
 5.5: 5.2132664,
 5.8: 5.300041,
 5.0: 4.4992857,
 6.3: 6.226211,
 6.5: 6.24,
 6.8: 6.612434,
 6.0: 5.6642623,
 7.3: 7.0475945,
 7.5: 7.2397704,
 7.8: 7.537892,
 7.0: 6.865918,
 8.3: 7.9686,
 8.5: 8.1226425,
 8.8: 8.388034,
 8.0: 7.479986,
 9.3: 9.252595,
 9.5: 9.415325,
 9.8: 9.579801,
 9.0: 8.883575}

In [5]:
errors = mean_absolute_error(validation_labels, preds)
errors

0.24936785363569497

In [9]:
list(dict.values())

['Predction',
 0.5912309,
 0.94844514,
 0.94844514,
 0.3405181,
 1.3779117,
 1.7255563,
 1.9736549,
 1.081826,
 9.2741375,
 2.2617373,
 2.4233904,
 2.70203,
 2.0484233,
 3.2267604,
 3.4911416,
 3.7960181,
 2.8444428,
 3.9012382,
 3.9978817,
 4.3437223,
 3.728656,
 4.883539,
 5.2132664,
 5.300041,
 4.4992857,
 6.226211,
 6.24,
 6.612434,
 5.6642623,
 7.0475945,
 7.2397704,
 7.537892,
 6.865918,
 7.9686,
 8.1226425,
 8.388034,
 7.479986,
 9.252595,
 9.415325,
 9.579801,
 8.883575]